In [ ]:
# Class 8 - Lecture - More on Modelling

In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from statsmodels.api import OLS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
# Read CSV file
bikes = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/bikes_sharing.csv')

In [4]:
# Bike Share for Washington DC
bikes.head()

# Season = is season for bike riding or not
# Holiday = 1 is a holiday
# Working Day
# Weather = 1 - categorical - severity
# Temp in Celsuis
# Atemp - feels like 
# Wind speed - speed of win
# Casual - people who are generally walking by and want
# Registered = number of ppl who have accounts, annual subscription
# Count = total of casual and regular users
# Hourly rental of bike rentals, at a given hour

# Influencing factor of bike rental, influence by humidity, weather, temperature

# Count - we are predicting the count rentals
# Make sure the downtown rental speed

# Casual renters - fun thing, tourists, occassion with friends/family, probably not on rush hour
# Registered - people who live in the city, such as commuters

# Probably rain or shine




,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1
